In [1]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent.parent) +'/')
from utils.visualize import img_grid

from torchvision import transforms 

import numpy as np
import pandas as pd
import torchio as tio
import nibabel as nib

import ipywidgets as ipyw
import matplotlib.pyplot as plt
import SimpleITK as sitk
import torch
from torch import nn
from  torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from torchmetrics.image.fid import FrechetInceptionDistance
import scipy
from sklearn import metrics
import json 

In [2]:
macaw_path = str(Path(os.getcwd()).parent.parent)
ukbb_path = '/home/erik.ohara/UKBB'
generated_path = '/work/forkert_lab/erik/MACAW/cf_images/PCA_sklearn_five_diff_res_denormalized'
crop_path = '/work/forkert_lab/erik/T1_cropped'
output_path = '/work/forkert_lab/erik/MACAW/cf_images/PCA_sklearn_five_diff_res_hm'
z_initial = 41
z_fim = 140
nsamples = 5
# histogram matching parameters
m_number_histogram_levels = 500
m_number_match_points = 50
m_threshold_mean_intensity = False

In [3]:
data_path = ukbb_path + '/ukbb_img.csv'
df = pd.read_csv(data_path,low_memory=False)

In [4]:
# Getting the generated images
subjects_eid = np.array([])
cf_age = np.array([])
cf_sex = np.array([])
images_generated = []
files_generated = []
for file in tqdm(os.listdir(generated_path)):
    if '.nii' in file:
        files_generated.append(file)
        splits = file.split("_")
        subject_eid = int(splits[0])
        age = splits[1]
        sex = splits[2].split(".")[0]
        subjects_eid = np.append(subjects_eid,subject_eid)
        cf_age = np.append(cf_age, age)
        cf_sex = np.append(cf_sex, sex)
        image_load = sitk.ReadImage(os.path.join(generated_path, file))
        #image_load = nib.load(os.path.join(generated_path, file)).get_fdata()
        images_generated.append(image_load)
print(len(images_generated))

  0%|          | 0/2370 [00:00<?, ?it/s]

2370


In [5]:
# Getting the cropped original images
images_original = []
real_age = []
real_sex = []
for individual in tqdm(subjects_eid):
    individual = int(individual)
    real_age.append(df[df["eid"] == individual]["Age"].item())
    real_sex.append(df[df["eid"] == individual]["Sex"].item())
    #image_load = nib.load(os.path.join(ukbb_T1_warped, str(individual) + ".nii.gz")).get_fdata()
    image_load = sitk.ReadImage(os.path.join(crop_path, str(individual) + ".nii.gz"))
    images_original.append(image_load)
print(len(images_original))

  0%|          | 0/2370 [00:00<?, ?it/s]

2370


In [6]:
stats_filter = sitk.StatisticsImageFilter()
stats_filter.Execute(images_original[1])
print(stats_filter.GetMean())
print(stats_filter.GetMaximum())
print(stats_filter.GetMinimum())

625.5206582363726
2556.879638671875
0.0


In [7]:
stats_filter = sitk.StatisticsImageFilter()
stats_filter.Execute(images_generated[1])
print(stats_filter.GetMean())
print(stats_filter.GetMaximum())
print(stats_filter.GetMinimum())

626.9145094131126
2557.655029296875
-70.09840393066406


In [8]:
stats_filter = sitk.StatisticsImageFilter()
stats_filter.Execute(images_original[130])
print(stats_filter.GetMean())
print(stats_filter.GetMaximum())
print(stats_filter.GetMinimum())

586.054588511445
1870.926025390625
-0.4781421422958374


In [9]:
stats_filter = sitk.StatisticsImageFilter()
stats_filter.Execute(images_generated[130])
print(stats_filter.GetMean())
print(stats_filter.GetMaximum())
print(stats_filter.GetMinimum())

580.2107605066752
1872.2847900390625
-57.28786087036133


In [10]:
images_generated[0].GetSize()[2]

100

In [11]:
for idx,each_image in tqdm(enumerate(images_generated)):
    for z_slice in range(each_image.GetSize()[2]):
        reference_slice = images_original[idx][:,:,z_slice]
        hist_filter = sitk.HistogramMatchingImageFilter()
        # Set filter parameters
        hist_filter.SetNumberOfHistogramLevels(m_number_histogram_levels) # number of bins for the histograms
        hist_filter.SetNumberOfMatchPoints(m_number_match_points) # number of quantile values to be matched
        hist_filter.SetThresholdAtMeanIntensity(m_threshold_mean_intensity)

        each_image[:,:,z_slice] = hist_filter.Execute(each_image[:,:,z_slice], reference_slice)

0it [00:00, ?it/s]

In [12]:
stats_filter = sitk.StatisticsImageFilter()
stats_filter.Execute(images_generated[1])
print(stats_filter.GetMean())
print(stats_filter.GetMaximum())
print(stats_filter.GetMinimum())

625.9115528246348
2556.879638671875
0.0


In [13]:
stats_filter = sitk.StatisticsImageFilter()
stats_filter.Execute(images_generated[130])
print(stats_filter.GetMean())
print(stats_filter.GetMaximum())
print(stats_filter.GetMinimum())

586.4163811256072
1870.926025390625
-0.4781421422958374


In [14]:
for idx,each_image in tqdm(enumerate(images_generated)):
    sitk.WriteImage(each_image, os.path.join(output_path, files_generated[idx]))

0it [00:00, ?it/s]